In [ ]:
!pip install deepl

In [ ]:
import os
import json
import deepl
from google.colab import files

# Set your DeepL API key
api_key = '25c28303-93e1-45ec-9624-18f09d8030b1:fx'  # Replace with your DeepL API key
translator = deepl.Translator(api_key)

# Upload the files
uploaded = files.upload()

# Function to translate text using DeepL
def translate_text(text):
    # Make sure the text is not empty or None
    if text:
        try:
            result = translator.translate_text(text, target_lang='EN')
            return result.text
        except Exception as e:
            print(f"Error during translation: {e}")
            return text
    return text

# Function to process and translate a JSON file
def process_json_file(file_path):
    with open(file_path, 'r', encoding='utf-8') as f:
        data = json.load(f)
    
    # Recursively translate all Chinese text
    def recursive_translate(data):
        if isinstance(data, dict):
            for key, value in data.items():
                data[key] = recursive_translate(value)
        elif isinstance(data, list):
            for i in range(len(data)):
                data[i] = recursive_translate(data[i])
        elif isinstance(data, str):
            # Translate only if the string contains Chinese characters
            if any('\u4e00' <= c <= '\u9fff' for c in data):  # Check for Chinese characters
                return translate_text(data)
        return data

    translated_data = recursive_translate(data)

    # Save the translated JSON back to file
    with open(file_path, 'w', encoding='utf-8') as f:
        json.dump(translated_data, f, ensure_ascii=False, indent=4)

# Process all uploaded JSON files
for filename in uploaded.keys():
    if filename.endswith('.json'):
        print(f"Processing {filename}...")
        process_json_file(filename)

print("Translation completed!")
